# Processing of Travel StackExchange Data

## Introduction
This lab work performs data processing on Travel StackExchange data to compare Python (Pandas) and SQL approaches.


## Data Preparation
### Import Libraries and Data

In [42]:
import os
import sqlite3
import tempfile

from olga_grigorieva_assignment_2 import *

### Load Data into DataFrame and SQL Database
# Load data into Pandas DataFrames

In [43]:
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv")
Users = pd.read_csv("travel_stackexchange_com/Users.csv")
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv")
PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv")

# Initialize SQL database and load data into it

In [44]:
baza = os.path.join(tempfile.mkdtemp(), 'example.db')
conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)

29019

# Select data from SQL database using SQL query

In [45]:
def solution_1_sql(conn):
    query = """
        SELECT Location, COUNT(*) AS Count 
        FROM (
            SELECT Posts.OwnerUserId, Users.Id, Users.Location 
            FROM Users
            JOIN Posts ON Users.Id = Posts.OwnerUserId
        )
        WHERE Location NOT IN ('') 
        GROUP BY Location
        ORDER BY Count DESC
        LIMIT 10
    """
    return pd.read_sql_query(query, conn)


def solution_2_sql(conn):
    query = """
        SELECT Posts.Title, RelatedTab.NumLinks 
        FROM (
            SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks 
            FROM PostLinks
            GROUP BY RelatedPostId
            ) AS RelatedTab
        JOIN Posts ON RelatedTab.PostId=Posts.Id 
        WHERE Posts.PostTypeId=1
        ORDER BY NumLinks DESC
    """
    return pd.read_sql_query(query, conn)


def solution_3_sql(conn):
    query = """
        SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location 
        FROM (
                SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount, CmtTotScr.CommentsTotalScore
                FROM (
                        SELECT PostId, SUM(Score) AS CommentsTotalScore 
                        FROM Comments
                        GROUP BY PostId
                    ) AS CmtTotScr
                JOIN Posts ON Posts.Id = CmtTotScr.PostId 
                WHERE Posts.PostTypeId=1
            ) AS PostsBestComments
        JOIN Users ON PostsBestComments.OwnerUserId = Users.Id 
        ORDER BY CommentsTotalScore DESC
        LIMIT 10
    """
    return pd.read_sql_query(query, conn)


def solution_4_sql(conn):
    query = """
        SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes 
        FROM (
            SELECT * 
            FROM (
                SELECT COUNT(*) as AnswersNumber, OwnerUserId 
                FROM Posts
                WHERE PostTypeId = 2
                GROUP BY OwnerUserId
                ) AS Answers 
            JOIN
                (
                    SELECT COUNT(*) as QuestionsNumber, OwnerUserId 
                    FROM Posts
                    WHERE PostTypeId = 1
                    GROUP BY OwnerUserId
                ) AS Questions
            ON Answers.OwnerUserId = Questions.OwnerUserId 
            WHERE AnswersNumber > QuestionsNumber
            ORDER BY AnswersNumber DESC
            LIMIT 5
            ) AS PostsCounts 
        JOIN Users
        ON PostsCounts.OwnerUserId = Users.Id
    """
    return pd.read_sql_query(query, conn)


def solution_5_sql(conn):
    query = """
        SELECT Questions.Id,
            Questions.Title,
            BestAnswers.MaxScore,
            Posts.Score AS AcceptedScore, 
            BestAnswers.MaxScore-Posts.Score AS Difference
        FROM (
            SELECT Id, ParentId, MAX(Score) AS MaxScore 
            FROM Posts
            WHERE PostTypeId==2
            GROUP BY ParentId
            ) AS BestAnswers
        JOIN (
            SELECT * FROM Posts
            WHERE PostTypeId==1 
            ) AS Questions
        ON Questions.Id=BestAnswers.ParentId
        JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id WHERE Difference>50
        ORDER BY Difference DESC
    """
    return pd.read_sql_query(query, conn)

# Function for result comparing

In [46]:
def compare_results(task_name, pandas_result, sql_result):
    comparison = pandas_result.equals(sql_result)
    print(f"Task {task_name} Comparison: {comparison}")

## Task 1: Analyzing User Locations
### Task Summary
Determine the best user locations based on post count.

### Pandas Implementation

In [47]:
pandas_result_1 = solution_1(Posts, Users)

### SQL Implementation

In [48]:
sql_result_1 = solution_1_sql(conn)

### Comparison

In [49]:
print("Task 1 Report:")
print("Pandas Solution:")
print(pandas_result_1.head())
print("\nSQL Solution:")
print(sql_result_1.head())
compare_results(1, pandas_result_1, sql_result_1)

Task 1 Report:
Pandas Solution:
                        Location  Count
0      Christchurch, New Zealand   2795
1                   New York, NY   1872
2         London, United Kingdom   1711
3                             UK   1707
4  Sunshine Coast QLD, Australia   1558

SQL Solution:
                        Location  Count
0      Christchurch, New Zealand   2795
1                   New York, NY   1872
2         London, United Kingdom   1711
3                             UK   1707
4  Sunshine Coast QLD, Australia   1558
Task 1 Comparison: True


## Task 2: Most Linked Posts
### Task Summary
Identify the most referenced posts in the forum.

### Pandas Implementation

In [50]:
pandas_result_2 = solution_2(Posts, PostLinks)

### SQL Implementation

In [51]:
sql_result_2 = solution_2_sql(conn)

### Comparison

In [52]:
print("Task 2 Report:")
print("Pandas Solution:")
print(pandas_result_2.head())
print("\nSQL Solution:")
print(sql_result_2.head())
compare_results(2, pandas_result_2, sql_result_2)

Task 2 Report:
Pandas Solution:
                                               Title  NumLinks
0  Is there a way to find out if I need a transit...      1157
1  Do I need a visa to transit (or layover) in th...      1012
2  Should I submit bank statements when applying ...       669
3  UK visa refusal on V 4.2 a + c (and sometimes ...       523
4  Should my first trip be to the country which i...       461

SQL Solution:
                                               Title  NumLinks
0  Is there a way to find out if I need a transit...      1157
1  Do I need a visa to transit (or layover) in th...      1012
2  Should I submit bank statements when applying ...       669
3  UK visa refusal on V 4.2 a + c (and sometimes ...       523
4  Should my first trip be to the country which i...       461
Task 2 Comparison: True


## Task 3: The best Posts by Comment Engagement
### Task Summary
Evaluate posts with the highest total score from comments, along with associated user details.

### Pandas Implementation

In [53]:
pandas_result_3 = solution_3(Posts, Users, Comments)

### SQL Implementation

In [54]:
sql_result_3 = solution_3_sql(conn)

### Comparison

In [55]:
print("Task 3 Report:")
print("Pandas Solution:")
print(pandas_result_3.head())
print("\nSQL Solution:")
print(sql_result_3.head())
compare_results(3, pandas_result_3, sql_result_3)

Task 3 Report:
Pandas Solution:
                                               Title  CommentCount  ViewCount  \
0  Boss is asking for passport, but it has a stam...            24    61309.0   
1  How to intentionally get denied entry to the U...            37    72856.0   
2  Can I wear a bulletproof vest while traveling ...            26    23467.0   
3  OK we're all adults here, so really, how on ea...            29   113001.0   
4  Being separated from one's young children on a...            36    14308.0   

   CommentsTotalScore  DisplayName  Reputation                       Location  
0                 630        Megha        1679                            NaN  
1                 618    user11743        1997                            NaN  
2                 403       Ulkoma        9190         London, United Kingdom  
3                 388  hippietrail       75994  Sunshine Coast QLD, Australia  
4                 358  Emma-louise         611                            NaN  



## Task 4: User Activity Comparison
### Task Summary
Identify the best 5 users with more answers than questions posted, including their reputations and locations.

### Pandas Implementation

In [56]:
pandas_result_4 = solution_4(Posts, Users)

### SQL Implementation

In [57]:
sql_result_4 = solution_4_sql(conn)

### Comparison

In [58]:
print("Task 4 Report:")
print("Pandas Solution:")
print(pandas_result_4.head())
print("\nSQL Solution:")
print(sql_result_4.head())
compare_results(4, pandas_result_4, sql_result_4)

Task 4 Report:
Pandas Solution:
   DisplayName  QuestionsNumber  AnswersNumber                   Location  \
0    Mark Mayo              344           1968  Christchurch, New Zealand   
1        phoog               12           1690               New York, NY   
2      Relaxed                8           1506                        NaN   
3  lambshaanxy              117           1466                        NaN   
4     Crazydre              172           1163                        NaN   

   Reputation  UpVotes  DownVotes  
0      157193    16082       1661  
1      120317     8767        567  
2       99185    11268        606  
3       98555    15021        275  
4       73180     2263        132  

SQL Solution:
   DisplayName  QuestionsNumber  AnswersNumber                   Location  \
0    Mark Mayo              344           1968  Christchurch, New Zealand   
1        phoog               12           1690               New York, NY   
2      Relaxed                8           1

## Task 5: Analyzing Question and Answer Scores
### Task Summary
Compare the highest answer score to the accepted answer score for questions, and identify significant discrepancies.

### Pandas Implementation

In [59]:
pandas_result_5 = solution_5(Posts)

### SQL Implementation

In [60]:
sql_result_5 = solution_5_sql(conn)

### Comparison

In [61]:
print("Task 5 Report:")
print("Pandas Solution:")
print(pandas_result_5.head())
print("\nSQL Solution:")
print(sql_result_5.head())
compare_results(5, pandas_result_5, sql_result_5)

Task 5 Report:
Pandas Solution:
       Id                                              Title  MaxScore  \
0  137989               How do I minimise waste on a flight?       125   
1  133636  What Happens when Passenger Refuses to Fly Boe...       113   
2   99713  In Germany, what are you supposed to do if you...       133   
3   76737  Why do many countries in the world still requi...       120   
4   81376  What is way to eat rice with hands in front of...       125   

   AcceptedScore  Difference  
0             15         110  
1             12         101  
2             38          95  
3             33          87  
4             39          86  

SQL Solution:
       Id                                              Title  MaxScore  \
0  137989               How do I minimise waste on a flight?       125   
1  133636  What Happens when Passenger Refuses to Fly Boe...       113   
2   99713  In Germany, what are you supposed to do if you...       133   
3   76737  Why do many cou

## Conclusion
This assignment involves data processing of a simplified dataset from Travel StackExchange using Pandas and SQL in Python.
I performed tasks such as data loading, database creation and result comparison for various queries. 
The dataset includes Posts, Users, Comments and PostLinks. 
Solutions have been implemented in both Pandas and SQL, ensuring equivalent results. 
The final report, created in a Jupyter notebook, have done well-structured with Markdown formatting, highlighting each task and including comparisons of results.